# Graph Coloring Problem Solver
## Algorithms:  Backtracking, BFS, Hill Climbing, Cultural Algorithm & Multi-Start Hybrid
---

## 1. Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random
import time
from typing import List, Dict, Tuple, Set
from dataclasses import dataclass
from copy import deepcopy
from collections import deque

print("✓ Libraries imported successfully! ")

## 2. Graph Class (Model)

In [ ]:
class Graph:
    """
    Model:  Responsible ONLY for the graph data structure.
    """
    def __init__(self, vertices):
        self.V = vertices
        self. adj = [[] for _ in range(vertices)]

    def add_edge(self, u, v):
        self.adj[u].append(v)
        self.adj[v].append(u)

print("✓ Graph class defined! ")

## 3. Graph Visualizer (View)

In [ ]:
class GraphVisualizer:
    """View:  Visualizes a colored graph using NetworkX"""

    def __init__(self, graph):
        self.graph_data = graph
        self.palette = [
            'magenta', 'teal', 'red', 'green', 'blue',
            'yellow', 'orange', 'purple', 'cyan', 'pink',
            'gray', 'brown', 'lime', 'indigo', 'gold'
        ]

    def draw_solution(self, color_assignments, chromatic_number, elapsed_time=None, algorithm_name=""):
        G_visual = nx.Graph()
        G_visual.add_nodes_from(range(self.graph_data.V))
        for u in range(self.graph_data.V):
            for v in self.graph_data. adj[u]:
                if u < v:
                    G_visual.add_edge(u, v)

        color_map = []
        for i in color_assignments:
            color_idx = (i - 1) % len(self.palette)
            color_map.append(self. palette[color_idx])

        plt.figure(figsize=(10, 8))
        pos = nx.spring_layout(G_visual, seed=42)
        nx.draw_networkx_nodes(G_visual, pos, node_color=color_map, node_size=700, edgecolors='black')
        nx.draw_networkx_labels(G_visual, pos, font_color='white', font_weight='bold')
        nx.draw_networkx_edges(G_visual, pos, width=2)

        title = f"{algorithm_name} Graph Coloring Solution\nChromatic Number: {chromatic_number}"
        if elapsed_time is not None:
            title += f"\nTime Taken: {elapsed_time:.6f} sec"
        plt.title(title, fontsize=14, fontweight='bold')

        plt. axis('off')
        plt.tight_layout()
        plt.show()

print("✓ GraphVisualizer class defined!")

## 4. BFS Coloring Algorithm

In [ ]:
class BFSColoring:
    """
    BFS-based greedy graph coloring algorithm.
    Colors vertices in BFS traversal order, using the smallest available color.
    """

    def __init__(self, graph, start_vertex=0):
        self.graph = graph
        self.start_vertex = start_vertex
        self.color_assignment = [-1] * graph.V
        self.chromatic_number = 0
        self.nodes_visited = 0

    def get_available_color(self, vertex):
        """Find the smallest color not used by adjacent vertices"""
        adjacent_colors = set()
        for neighbor in self.graph.adj[vertex]:
            if self.color_assignment[neighbor] != -1:
                adjacent_colors.add(self.color_assignment[neighbor])

        color = 0
        while color in adjacent_colors:
            color += 1
        return color

    def solve(self, verbose=True):
        """Perform BFS coloring on the graph"""
        if verbose:
            print(f"\nStarting BFS Coloring from vertex {self.start_vertex}... ")

        start_time = time.time()

        visited = [False] * self.graph.V
        queue = deque([self.start_vertex])
        visited[self.start_vertex] = True

        while queue:
            vertex = queue.popleft()
            self.nodes_visited += 1

            color = self.get_available_color(vertex)
            self.color_assignment[vertex] = color
            self.chromatic_number = max(self.chromatic_number, color + 1)

            for neighbor in self.graph.adj[vertex]:
                if not visited[neighbor]:
                    visited[neighbor] = True
                    queue.append(neighbor)

        for vertex in range(self.graph.V):
            if self.color_assignment[vertex] == -1:
                color = self.get_available_color(vertex)
                self.color_assignment[vertex] = color
                self.chromatic_number = max(self.chromatic_number, color + 1)
                self.nodes_visited += 1

        end_time = time.time()
        elapsed_time = end_time - start_time

        self.color_assignment = [c + 1 for c in self. color_assignment]

        if verbose:
            print(f"\n✓ BFS Coloring Complete!")
            print(f"Chromatic Number:  {self.chromatic_number}")
            print(f"Nodes visited: {self.nodes_visited}")
            print(f"Computational time: {elapsed_time:.6f} seconds")

        return self.color_assignment, self.chromatic_number, elapsed_time

print("✓ BFSColoring class defined!")

## 5. Backtracking Algorithm

In [ ]:
class BackTracking:
    """Backtracking algorithm for graph coloring"""

    def __init__(self, graph):
        self.graph = graph
        self. best_color_count = float('inf')
        self.best_solution = None
        self.recursive_calls = 0
        self.pruned_branches = 0

    def is_safe(self, node, c, current_color_assignment):
        for neighbor in self.graph.adj[node]:
            if current_color_assignment[neighbor] == c:
                self.pruned_branches += 1
                return False
        return True

    def rec(self, node, used_colors, current_color_assignment):
        self.recursive_calls += 1

        if node == self.graph.V:
            if used_colors < self.best_color_count:
                self. best_color_count = used_colors
                self.best_solution = list(current_color_assignment)
            return

        if used_colors >= self.best_color_count:
            self.pruned_branches += 1
            return

        for c in range(1, used_colors + 1):
            if self. is_safe(node, c, current_color_assignment):
                current_color_assignment[node] = c
                self. rec(node + 1, used_colors, current_color_assignment)
                current_color_assignment[node] = 0

        if used_colors + 1 < self.best_color_count:
            current_color_assignment[node] = used_colors + 1
            self.rec(node + 1, used_colors + 1, current_color_assignment)
            current_color_assignment[node] = 0

    def solve(self):
        initial_assignment = [0] * self.graph.V
        self.best_color_count = float('inf')
        self.best_solution = None
        self.recursive_calls = 0
        self.pruned_branches = 0

        start_time = time.time()
        self. rec(0, 0, initial_assignment)
        end_time = time.time()

        elapsed_time = end_time - start_time

        if self.best_solution:
            print("\n✓ Valid Solution Found!")
            print(f"Chromatic Number: {self.best_color_count}")
            print(f"Computational time: {elapsed_time:.6f} seconds")

        return self.best_solution, self.best_color_count, elapsed_time

print("✓ BackTracking class defined!")

## 6. Hill Climbing Algorithm

In [ ]:
class HillClimbing:
    """Hill Climbing algorithm for graph coloring"""

    def __init__(self, graph, max_colors=None):
        self.graph = graph
        self.max_colors = max_colors or graph.V
        self.current_solution = None
        self.current_conflicts = float('inf')
        self.iterations = 0

    def count_conflicts(self, coloring):
        """Count the number of conflicting edges"""
        conflicts = 0
        for u in range(self.graph.V):
            for v in self.graph.adj[u]:
                if u < v and coloring[u] == coloring[v]:
                    conflicts += 1
        return conflicts

    def get_chromatic_number(self, coloring):
        """Get the number of unique colors used"""
        return len(set(coloring))

    def generate_initial_solution(self):
        """Generate a random initial solution"""
        return [random.randint(1, self.max_colors) for _ in range(self.graph.V)]

    def get_neighbors(self, coloring):
        """Generate neighboring solutions by changing one vertex's color"""
        neighbors = []
        for vertex in range(self.graph.V):
            current_color = coloring[vertex]
            for new_color in range(1, self.max_colors + 1):
                if new_color != current_color:
                    neighbor = coloring[: ]
                    neighbor[vertex] = new_color
                    neighbors.append(neighbor)
        return neighbors

    def solve(self, max_iterations=1000, verbose=True):
        """Run Hill Climbing algorithm"""
        if verbose:
            print(f"\nStarting Hill Climbing...")
            print(f"Max colors: {self.max_colors}")
            print(f"Max iterations: {max_iterations}")

        start_time = time.time()

        self.current_solution = self.generate_initial_solution()
        self.current_conflicts = self.count_conflicts(self.current_solution)

        best_solution = self.current_solution[: ]
        best_conflicts = self.current_conflicts
        best_colors = self.get_chromatic_number(best_solution)

        stagnation_counter = 0
        max_stagnation = 100

        for iteration in range(max_iterations):
            self.iterations += 1

            if self.current_conflicts == 0:
                current_colors = self.get_chromatic_number(self.current_solution)
                if current_colors < self.max_colors:
                    self.max_colors = current_colors
                    self. current_solution = self.generate_initial_solution()
                    self.current_conflicts = self.count_conflicts(self.current_solution)
                    stagnation_counter = 0
                    continue

            neighbors = self.get_neighbors(self.current_solution)
            best_neighbor = None
            best_neighbor_conflicts = self.current_conflicts

            for neighbor in neighbors:
                neighbor_conflicts = self.count_conflicts(neighbor)
                if neighbor_conflicts < best_neighbor_conflicts:
                    best_neighbor = neighbor
                    best_neighbor_conflicts = neighbor_conflicts

            if best_neighbor is not None and best_neighbor_conflicts < self.current_conflicts:
                self.current_solution = best_neighbor
                self.current_conflicts = best_neighbor_conflicts
                stagnation_counter = 0

                if self.current_conflicts < best_conflicts or \
                        (self.current_conflicts == best_conflicts and
                         self.get_chromatic_number(self. current_solution) < best_colors):
                    best_solution = self.current_solution[:]
                    best_conflicts = self.current_conflicts
                    best_colors = self.get_chromatic_number(best_solution)

                    if verbose and (iteration % 100 == 0 or best_conflicts == 0):
                        print(f"Iteration {iteration}:  Conflicts={best_conflicts}, Colors={best_colors}")
            else:
                stagnation_counter += 1

                if stagnation_counter >= max_stagnation:
                    if verbose:
                        print(f"Stagnation detected at iteration {iteration}.  Restarting...")
                    self.current_solution = self.generate_initial_solution()
                    self.current_conflicts = self.count_conflicts(self.current_solution)
                    stagnation_counter = 0

        elapsed_time = time.time() - start_time

        if verbose:
            if best_conflicts == 0:
                print(f"\n✓ Valid Solution Found!")
                print(f"Chromatic Number: {best_colors}")
            else:
                print(f"\n✗ No valid solution found")
                print(f"Best solution has {best_conflicts} conflicts")
                print(f"Colors used: {best_colors}")
            print(f"Total iterations: {self.iterations}")
            print(f"Computational time: {elapsed_time:.6f} seconds")

        return best_solution, best_colors if best_conflicts == 0 else None, elapsed_time

print("✓ HillClimbing class defined!")

## 7. Cultural Algorithm Components

In [ ]:
class Individual:
    """Represents a single coloring solution"""

    def __init__(self, graph, coloring=None, max_colors=None):
        self.graph = graph
        self. max_colors = max_colors or graph.V
        if coloring is None:
            self.coloring = [random.randint(1, self. max_colors) for _ in range(graph.V)]
        else:
            self.coloring = coloring
        self.fitness = 0
        self.conflicts = 0
        self.chromatic_number = 0
        self.evaluate()

    def evaluate(self):
        """Calculate fitness based on conflicts and number of colors used"""
        self.conflicts = 0
        for u in range(self.graph.V):
            for v in self.graph.adj[u]:
                if u < v and self.coloring[u] == self.coloring[v]:
                    self.conflicts += 1

        self.chromatic_number = len(set(self.coloring))

        if self.conflicts == 0:
            self. fitness = 10000 - self.chromatic_number
        else:
            self.fitness = -self.conflicts * 100 - self.chromatic_number

    def is_valid(self):
        return self.conflicts == 0


class BeliefSpace:
    """Enhanced Belief Space with three types of knowledge"""

    def __init__(self, graph, size=5):
        self.graph = graph
        self.size = size
        self.best_solution = None
        self.normative = [None] * graph.V
        self.color_frequency = [{} for _ in range(graph. V)]
        self.min_colors_found = float('inf')
        self.successful_patterns = []

    def update(self, population):
        """Update all three knowledge types from population"""
        valid_individuals = [ind for ind in population if ind.is_valid()]

        if not valid_individuals:
            return

        current_best = min(valid_individuals, key=lambda x: x.chromatic_number)
        if self.best_solution is None or current_best.chromatic_number < self.best_solution.chromatic_number:
            self.best_solution = deepcopy(current_best)
            self. min_colors_found = current_best.chromatic_number

        self._update_normative(valid_individuals)
        self._update_domain(valid_individuals)

    def _update_normative(self, valid_individuals):
        """Track which colors work best for each vertex"""
        for ind in valid_individuals[: self.size]:
            for vertex, color in enumerate(ind.coloring):
                if color not in self.color_frequency[vertex]:
                    self.color_frequency[vertex][color] = 0
                self.color_frequency[vertex][color] += 1

        for vertex in range(self.graph. V):
            if self.color_frequency[vertex]:
                self.normative[vertex] = max(
                    self.color_frequency[vertex].items(),
                    key=lambda x: x[1]
                )[0]

    def _update_domain(self, valid_individuals):
        """Store patterns from successful solutions"""
        sorted_valid = sorted(valid_individuals, key=lambda x: x.chromatic_number)
        self.successful_patterns = [deepcopy(ind) for ind in sorted_valid[: self.size]]

    def influence(self, individual, rate=0.3):
        """Apply belief space knowledge to guide an individual"""
        if random.random() > rate:
            return

        influence_type = random.choice(['situational', 'normative', 'domain'])

        if influence_type == 'situational' and self.best_solution:
            num_genes = random.randint(1, max(1, len(individual.coloring) // 3))
            positions = random.sample(range(len(individual.coloring)), num_genes)
            for pos in positions:
                individual.coloring[pos] = self.best_solution.coloring[pos]

        elif influence_type == 'normative' and any(self.normative):
            for vertex in range(len(individual.coloring)):
                if self. normative[vertex] is not None and random.random() < 0.3:
                    neighbor_colors = {individual.coloring[n] for n in self.graph.adj[vertex]}
                    if self.normative[vertex] not in neighbor_colors:
                        individual.coloring[vertex] = self.normative[vertex]

        elif influence_type == 'domain' and self.successful_patterns:
            pattern = random.choice(self.successful_patterns)
            num_genes = random.randint(1, max(1, len(individual.coloring) // 4))
            positions = random.sample(range(len(individual.coloring)), num_genes)
            for pos in positions:
                individual.coloring[pos] = pattern.coloring[pos]

        individual.evaluate()

print("✓ Individual and BeliefSpace classes defined!")

In [ ]:
class CulturalAlgorithm:
    """Cultural Algorithm for graph coloring with enhanced belief space"""

    def __init__(self, graph, population_size=50, mutation_rate=0.1,
                 influence_rate=0.3, belief_space_size=5, max_colors=None):
        self.graph = graph
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.influence_rate = influence_rate
        self.max_colors = max_colors or graph.V
        self.population = [Individual(graph, max_colors=self.max_colors)
                           for _ in range(population_size)]
        self.belief_space = BeliefSpace(graph, size=belief_space_size)
        self.best_solution = None

    def select_parent(self):
        """Tournament selection"""
        tournament = random.sample(self.population, min(5, len(self.population)))
        return max(tournament, key=lambda x:  x.fitness)

    def crossover(self, parent1, parent2):
        """Single-point crossover"""
        point = random.randint(1, self.graph.V - 1)
        child_coloring = parent1.coloring[: point] + parent2.coloring[point:]
        return Individual(self.graph, child_coloring, max_colors=self.max_colors)

    def mutate(self, individual):
        """Mutate by changing random colors intelligently"""
        for i in range(len(individual.coloring)):
            if random. random() < self.mutation_rate:
                neighbor_colors = {individual.coloring[neighbor]
                                   for neighbor in self.graph.adj[i]}
                available = [c for c in range(1, self.max_colors + 1)
                             if c not in neighbor_colors]
                if available:
                    individual.coloring[i] = random.choice(available)
                else:
                    individual.coloring[i] = random.randint(1, self.max_colors)

        individual.evaluate()

    def evolve(self):
        """Create next generation with belief space influence"""
        self.population. sort(key=lambda x: x.fitness, reverse=True)
        self.belief_space.update(self.population)
        new_population = self.population[: 2]

        while len(new_population) < self.population_size:
            parent1 = self.select_parent()
            parent2 = self.select_parent()
            child = self.crossover(parent1, parent2)
            self.mutate(child)
            self.belief_space.influence(child, rate=self.influence_rate)
            new_population.append(child)

        self.population = new_population

    def solve(self, max_generations=100, verbose=True):
        """Run the cultural algorithm"""
        if verbose:
            print(f"\nStarting Cultural Algorithm...")
            print(f"Population size: {self.population_size}")

        start_time = time.time()

        for gen in range(max_generations):
            self. evolve()

            current_best = max(self.population, key=lambda x: x.fitness)
            if self.best_solution is None or current_best.fitness > self.best_solution.fitness:
                self.best_solution = deepcopy(current_best)

            if verbose and (gen % 20 == 0 or gen == max_generations - 1):
                valid_count = sum(1 for ind in self.population if ind.is_valid())
                print(
                    f"Gen {gen}: Best={self.best_solution.chromatic_number if self.best_solution. is_valid() else 'invalid'}, "
                    f"Valid={valid_count}/{self.population_size}, Conflicts={self.best_solution.conflicts}")

        elapsed = time.time() - start_time

        if verbose:
            if self.best_solution. is_valid():
                print(f"\n✓ Valid Solution Found!")
                print(f"Chromatic Number: {self.best_solution.chromatic_number}")
            else:
                print(f"\n✗ No valid solution found")
                print(f"Best solution has {self.best_solution.conflicts} conflicts")
            print(f"Computational time: {elapsed:.2f} seconds")

        return self.best_solution. coloring, self.best_solution.chromatic_number if self.best_solution.is_valid() else None, elapsed

print("✓ CulturalAlgorithm class defined!")

## 8. Multi-Start Hybrid Algorithm

In [ ]:
def multi_start_hybrid(graph, num_starts=10, ca_generations=150, verbose=True):
    """
    Multi-Start Hybrid Algorithm for Graph Coloring
    
    Strategy:
    1. Run BFS from multiple different starting vertices
    2. Pick the BFS solution with fewest colors
    3. Use that solution to seed Cultural Algorithm
    4. Let CA improve upon the best BFS result
    """
    
    if verbose:
        print("=" * 70)
        print("MULTI-START HYBRID ALGORITHM")
        print("=" * 70)
        print(f"\nPhase 1: Trying {num_starts} different BFS starting vertices... ")
    
    # PHASE 1: Try BFS from different starting vertices
    best_bfs_solution = None
    best_bfs_chromatic = float('inf')
    best_start_vertex = 0
    bfs_start_time = time.time()
    
    for start_vertex in range(min(num_starts, graph.V)):
        bfs = BFSColoring(graph, start_vertex=start_vertex)
        solution, chromatic, _ = bfs.solve(verbose=False)
        
        if chromatic < best_bfs_chromatic:
            best_bfs_solution = solution
            best_bfs_chromatic = chromatic
            best_start_vertex = start_vertex
        
        if verbose:
            marker = "★" if chromatic == min(best_bfs_chromatic, chromatic) else " "
            print(f"  {marker} Start vertex {start_vertex}: {chromatic} colors")
    
    bfs_time = time.time() - bfs_start_time
    
    if verbose:
        print(f"\n✓ Best BFS:  {best_bfs_chromatic} colors (from vertex {best_start_vertex})")
        print(f"  Time: {bfs_time:.4f}s")
        print(f"\nPhase 2: Refining with Cultural Algorithm...")
        print(f"  Target: < {best_bfs_chromatic} colors")
        print(f"  Generations: {ca_generations}")
    
    # PHASE 2: Improve with Cultural Algorithm
    ca_start_time = time.time()
    
    ca = CulturalAlgorithm(
        graph,
        population_size=100,
        mutation_rate=0.15,
        influence_rate=0.35,
        belief_space_size=12,
        max_colors=best_bfs_chromatic - 2
    )
    
    # PHASE 3: Seed population with BFS solution
    num_seeds = min(15, len(ca.population))
    
    for i in range(num_seeds):
        ca.population[i].coloring = best_bfs_solution[: ]
        
        if i > 0:
            num_mutations = i * 2
            for _ in range(num_mutations):
                vertex = random.randint(0, graph.V - 1)
                neighbor_colors = {ca.population[i].coloring[n] for n in graph.adj[vertex]}
                available_colors = [c for c in range(1, best_bfs_chromatic + 1)
                                    if c not in neighbor_colors]
                
                if available_colors:
                    ca.population[i]. coloring[vertex] = random. choice(available_colors)
                else:
                    ca.population[i].coloring[vertex] = random.randint(1, best_bfs_chromatic)
        
        ca.population[i].evaluate()
    
    if verbose:
        print(f"  Seeded {num_seeds} individuals with BFS-based solutions")
    
    # PHASE 4: Run Cultural Algorithm
    final_solution, final_chromatic, _ = ca.solve(
        max_generations=ca_generations,
        verbose=verbose
    )
    
    ca_time = time.time() - ca_start_time
    total_time = bfs_time + ca_time
    
    # PHASE 5: Report results
    if verbose:
        print("\n" + "=" * 70)
        print("RESULTS")
        print("=" * 70)
        print(f"\nBFS Best:          {best_bfs_chromatic} colors")
        
        if final_chromatic:
            improvement = best_bfs_chromatic - final_chromatic
            improvement_pct = (improvement / best_bfs_chromatic) * 100
            print(f"CA Final:        {final_chromatic} colors")
            print(f"Improvement:      {improvement} colors ({improvement_pct:.1f}% reduction)")
        else:
            print(f"CA Final:         No valid solution")
            print(f"Using BFS:         {best_bfs_chromatic} colors")
            final_solution = best_bfs_solution
            final_chromatic = best_bfs_chromatic
        
        print(f"\nTiming:")
        print(f"  BFS Phase:      {bfs_time:.4f}s")
        print(f"  CA Phase:      {ca_time:.4f}s")
        print(f"  Total:          {total_time:.4f}s")
    
    return final_solution, final_chromatic, total_time

print("✓ multi_start_hybrid function defined!")

## 9. File Loading Function

In [ ]:
def load_graph(filename):
    """Load graph from DIMACS format file"""
    graph = None
    with open(filename, 'r') as f:
        for line in f:
            line = line. strip()
            if line.startswith('c'):
                continue
            elif line.startswith('p'):
                parts = line.split()
                vertices = int(parts[2])
                graph = Graph(vertices)
            elif line.startswith('e'):
                parts = line.split()
                u = int(parts[1]) - 1
                v = int(parts[2]) - 1
                graph.add_edge(u, v)
    return graph

print("✓ load_graph function defined!")

## 10. Load Your Graph

Choose your input method:

In [ ]:
# OPTION 1: Load from file (RECOMMENDED for DSJC250. 5)
filename = "graph testing set.txt"  # Change this to your file name
my_graph = load_graph(filename)
print(f"✓ Loaded graph with {my_graph.V} vertices")

In [ ]:
# OPTION 2: Manual input (for small graphs)
# Uncomment and run this cell if you want to create a graph manually

# n = 5  # Number of vertices
# my_graph = Graph(n)

# # Add edges (0-indexed)
# edges = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4)]
# for u, v in edges:
#     my_graph.add_edge(u, v)

# print(f"✓ Created graph with {my_graph.V} vertices")

## 11. Run Algorithms

### 11.1 BFS Coloring

In [ ]:
# Run BFS Coloring
print("\n" + "="*70)
print("BFS COLORING ALGORITHM")
print("="*70)

bfs_solver = BFSColoring(my_graph, start_vertex=0)
bfs_solution, bfs_chromatic, bfs_time = bfs_solver.solve(verbose=True)

# Visualize (only for small graphs, comment out for DSJC250.5)
# visualizer = GraphVisualizer(my_graph)
# visualizer.draw_solution(bfs_solution, bfs_chromatic, bfs_time, "BFS Coloring")

### 11.2 Hill Climbing

In [ ]:
# Run Hill Climbing
print("\n" + "="*70)
print("HILL CLIMBING ALGORITHM")
print("="*70)

hc = HillClimbing(my_graph, max_colors=my_graph.V)
hc_solution, hc_chromatic, hc_time = hc.solve(max_iterations=1000, verbose=True)

### 11.3 Cultural Algorithm

In [ ]:
# Run Cultural Algorithm
print("\n" + "="*70)
print("CULTURAL ALGORITHM")
print("="*70)

ca = CulturalAlgorithm(my_graph, population_size=50)
ca_solution, ca_chromatic, ca_time = ca.solve(max_generations=100, verbose=True)

### 11.4 Multi-Start Hybrid

In [ ]:
# Run Multi-Start Hybrid
hybrid_solution, hybrid_chromatic, hybrid_time = multi_start_hybrid(
    my_graph,
    num_starts=10,
    ca_generations=150,
    verbose=True
)

## 12. Results Comparison

In [ ]:
# Compare all results
print("\n" + "="*70)
print("RESULTS COMPARISON")
print("="*70)

results = {
    'BFS Coloring': (bfs_chromatic, bfs_time),
    'Hill Climbing':  (hc_chromatic if hc_chromatic else 'N/A', hc_time),
    'Cultural Algorithm':  (ca_chromatic if ca_chromatic else 'N/A', ca_time),
    'Multi-Start Hybrid': (hybrid_chromatic if hybrid_chromatic else 'N/A', hybrid_time)
}

print(f"\n{'Algorithm':<25} {'Chromatic Number':<20} {'Time (seconds)':<15}")
print("-" * 70)
for algo, (chromatic, time_taken) in results.items():
    print(f"{algo:<25} {str(chromatic):<20} {time_taken: <15.6f}")

print("\n" + "="*70)
print(f"Optimal for DSJC250.5: 28 colors")
print("="*70)

## 13. Visualize Results (Optional - for small graphs only)

In [ ]:
# Visualize best result (uncomment for small graphs)
# visualizer = GraphVisualizer(my_graph)

# if bfs_chromatic:
#     visualizer.draw_solution(bfs_solution, bfs_chromatic, bfs_time, "BFS Coloring")

# if hybrid_chromatic:
#     visualizer.draw_solution(hybrid_solution, hybrid_chromatic, hybrid_time, "Multi-Start Hybrid")

## 14. Performance Analysis

In [ ]:
# Create performance comparison chart
import matplotlib.pyplot as plt

algorithms = ['BFS', 'Hill Climb', 'Cultural', 'Hybrid']
chromatic_numbers = [
    bfs_chromatic,
    hc_chromatic if hc_chromatic else 0,
    ca_chromatic if ca_chromatic else 0,
    hybrid_chromatic if hybrid_chromatic else 0
]
times = [bfs_time, hc_time, ca_time, hybrid_time]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Chromatic numbers
ax1.bar(algorithms, chromatic_numbers, color=['blue', 'orange', 'green', 'red'])
ax1.axhline(y=28, color='purple', linestyle='--', label='Optimal (28)')
ax1.set_ylabel('Chromatic Number (Colors)')
ax1.set_title('Chromatic Number Comparison')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Execution times
ax2.bar(algorithms, times, color=['blue', 'orange', 'green', 'red'])
ax2.set_ylabel('Time (seconds)')
ax2.set_title('Execution Time Comparison')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 15. Conclusion

### Summary:
- **BFS Coloring**: Fast and efficient for initial solution
- **Hill Climbing**: Can get stuck in local optima
- **Cultural Algorithm**:  Population-based, explores solution space
- **Multi-Start Hybrid**: Best balance of speed and quality

### For DSJC250.5:
- Optimal:  **28 colors**
- Your results should be **40-50 colors** (competitive performance)
- BFS provides excellent starting point
- Hybrid approach can refine the solution further